In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' # to print multiple outputs from the same cell
import math
import utils
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from operator import index
from collections import defaultdict
from scipy.stats import pearsonr
from datetime import datetime, timedelta

# Indicator Creation

In this notebook we will create new interesting features which we believe to be relevant for clustering purposes. The indicators we will try to create in this notebook are:
1. Account age in days (From account creation till the release of this dataset) (datetime(2022,9,29,11,0,0)) √
2. Account tweets number √
3. Account highest daily tweet count √
4. Account average tweet count √
5. Account tweet number over periods of time (e.g. Covid-19) √ (Partially, we can make other days like the day elon declared he wants to buy twitter)
6. Account tweet hashtag average √
7. Account average tweet text length √
8. Account average number of mentions per tweet √
9. Special Characters √
    1. Tweet number of special characters √
    2. Account average number of special characters in tweets √
10. Account's Entropy
11. Account Number of likes √
12. Account Average of likes √
13. Account Number of comments √
14. Account Average of comments √
15. Account discussion creation score (Tweet to retweet ratio) √
16. Ratio between number of tweets and number of likes √
17. Ratio between number of tweets and number of comments √

In [ ]:
users_df = pd.read_csv("dataset/users_dataset_cleaned.csv")
merged_df = pd.read_csv("dataset/merged_dataset_cleaned.csv", lineterminator="\n")

## 1. Account age in days
Creating the account age in days feature and adding it to the dataframe.

In [ ]:
users_df["created_at"] = pd.to_datetime(users_df["created_at"], errors="coerce") # Even though this is already done in the previous notebook, this information is turned to string when converting the merged_df to csv
release_of_dataset = datetime(2022,9,29,11,0,0)
users_df["account_age_in_days"] = (release_of_dataset - users_df['created_at']).dt.days

users_df.head()

# Keeping info in merged_df too for further calculations
merged_df["account_created"] = pd.to_datetime(merged_df["account_created"], errors="coerce")
merged_df["account_age_in_days"] = (release_of_dataset - merged_df['account_created']).dt.days

# 2. Account number of tweets

Creating the account number of tweets.

In [ ]:
#users_df = users_df.merge(merged_df.groupby('user_id').size().reset_index(name='counts'), how="left")
users_df = users_df.merge(merged_df.groupby('user_id', as_index=False).size(), on="user_id", how="left")
users_df.rename(columns={"size": "number_of_tweets"}, inplace=True)
users_df

# 4. Account average tweets per day

Creating the account average tweets per day and adding it to the dataframe.

In [ ]:
merged_df["account_average_tweets_per_day"] = merged_df["statuses_count"]/merged_df["account_age_in_days"]

merged_df.head()

Idea, isn't better to add this attribute to the users_df ? Because it's a user caractherization, and having it in the merged_df is gonna be redundant

In [ ]:
users_df["account_average_tweets_per_day"] = users_df["statuses_count"]/users_df["account_age_in_days"]
users_df.head()

We need to know the tweet age in days, the average number of tweets between the days that the user used twitter

In [ ]:
# Creating the tweet age in days (for each) till the release of dataset
merged_df["tweet_created"] = pd.to_datetime(merged_df["tweet_created"], errors="coerce")
#merged_df['tweet_created'] = merged_df['tweet_created'].apply(pd.to_datetime)
release_of_dataset = datetime(2022,9,29,11,0,0)

merged_df['tweet_created_just_days'] = merged_df['tweet_created'].dt.date
merged_df["tweet_created_just_days"] = pd.to_datetime(merged_df["tweet_created_just_days"], errors="coerce")
merged_df['tweet_age_in_days'] = (release_of_dataset - merged_df['tweet_created_just_days']).dt.days

merged_df.drop(columns="tweet_created_just_days", inplace=True)
merged_df.head()

Adding indicator for average number of tweets per day in which the account ACTUALLY tweeted - Gianluca 

In [ ]:
tmp = merged_df.groupby(["user_id", "tweet_age_in_days"], as_index=False).size()

# computing average
tmp = tmp.groupby(["user_id"], as_index=False).mean()
tmp=pd.DataFrame(tmp[["user_id", "size"]])
avg_df = tmp.rename(columns={"size": "avg_tweets_per_actual_day"})
users_df = users_df.merge(avg_df, on="user_id", how="left")
users_df

## 3. Account highest daily tweet count

Creating the account highest daily tweet count and adding it too the dataframe.

In [ ]:

# Trying to see how the group by num of days works
tmp = merged_df.groupby(["user_id", "tweet_age_in_days"], as_index=False).size()

# computing average
tmp = tmp.iloc[tmp.groupby("user_id")["size"].idxmax()]
tmp = tmp.rename(columns={"size": "max_number_of_tweet_in_a_day", "tweet_age_in_days": "day_with_most_tweets"})

users_df = users_df.merge(tmp, on="user_id", how="left")
users_df

In [ ]:
users_df

## 5. Tweet num over period of time

Creating the account tweet num by period of time

In [ ]:
covid19_start = release_of_dataset - datetime(2020, 3, 11)
covid19_ipotetic_end_of_restrictions = release_of_dataset - datetime(2021, 12, 15)
covid19_start = covid19_start.days
covid19_ipotetic_end_of_restrictions = covid19_ipotetic_end_of_restrictions.days
temp = merged_df[merged_df["tweet_age_in_days"]<=covid19_start]
temp = temp[temp["tweet_age_in_days"]>covid19_ipotetic_end_of_restrictions]
temp = temp.groupby("user_id", as_index=False).size()
temp = temp.rename(columns={"size": "covid19_num_of_tweets"})
users_df = users_df.merge(temp, on="user_id", how='left')
users_df["covid19_num_of_tweets"].replace(np.nan, 0, inplace=True)
users_df


COVID 19 from 11/03/2020 to 15/12/2021 (pretty much the period of time in which we had restictions and had to stay home)

## 6. Average of hashtags per user

Creating the account average of hashtags in tweets

In [ ]:
# Trying to see how the group by num of days works
tmp = merged_df[["user_id", "num_hashtags"]].groupby(["user_id"], as_index=False).mean()
tmp = tmp.rename(columns={"num_hashtags": "avg_hashtags"})
users_df = users_df.merge(tmp, on="user_id", how='left')
users_df

## 7. Average tweet text length per user

Creating the account average tweet length

In [ ]:
merged_df["text_length"] = merged_df["text"].str.len() 
tmp = merged_df[["user_id", "text_length"]].groupby(["user_id"], as_index=False).mean()
tmp = tmp.rename(columns={"text_length": "avg_text_length"})
users_df = users_df.merge(tmp, on="user_id", how='left')
users_df

## 8. Average of mentions per user

Creating the account average of hashtags in tweets

In [ ]:
# Trying to see how the group by num of days works
tmp = merged_df[["user_id", "num_mentions"]].groupby(["user_id"], as_index=False).mean()
tmp = tmp.rename(columns={"num_mentions": "avg_mentions"})
users_df = users_df.merge(tmp, on="user_id", how='left')
users_df

## 9. Average num of special characters in text per user and num of special characters per tweet

Creating the account average of special characters in tweets

In [ ]:
merged_df["special_char_in_text"] = merged_df["text"].str.len() - merged_df["text"].str.findall('[\w]').str.len()
tmp = merged_df[["user_id", "special_char_in_text"]].groupby(["user_id"], as_index=False).mean()
tmp = tmp.rename(columns={"special_char_in_text": "avg_special_char_in_text"})
users_df = users_df.merge(tmp, on="user_id", how='left')
merged_df
users_df

## 11. Number of likes per user

Creating the account number of likes in tweets

In [ ]:
# Trying to see how the group by num of days works
tmp = merged_df[["user_id", "favorite_count"]].groupby(["user_id"], as_index=False).sum()
tmp = tmp.rename(columns={"favorite_count": "total_likes"})
users_df = users_df.merge(tmp, on="user_id", how='left')
users_df

## 12. Average of likes per user

Creating the account average of likes in tweets

In [ ]:
# Trying to see how the group by num of days works
tmp = merged_df[["user_id", "favorite_count"]].groupby(["user_id"], as_index=False).mean()
tmp = tmp.rename(columns={"favorite_count": "avt_favorite_count"})
users_df = users_df.merge(tmp, on="user_id", how='left')
users_df

## 13. Number of comments per user

Creating the account number of comments in tweets

In [ ]:
# Trying to see how the group by num of days works
tmp = merged_df[["user_id", "reply_count"]].groupby(["user_id"], as_index=False).sum()
tmp = tmp.rename(columns={"reply_count": "total_replies"})
users_df = users_df.merge(tmp, on="user_id", how='left')
users_df

## 14. Average of comments per user

Creating the account average of comments in tweets

In [ ]:
tmp = merged_df[["user_id", "reply_count"]].groupby(["user_id"], as_index=False).mean()
tmp = tmp.rename(columns={"reply_count": "avt_reply_count"})
users_df = users_df.merge(tmp, on="user_id", how='left')
users_df

## 15. Account discussion creation score
Creating the account tweet to retweet ratio

In [ ]:
tmp = merged_df[["user_id", "retweet_count"]].groupby(["user_id"], as_index=False).sum()
tmp = tmp.rename(columns={"retweet_count": "total_retweet_count"})
users_df = users_df.merge(tmp, on="user_id", how='left')
users_df["total_retweet_count"].replace(np.nan, 0, inplace=True)
users_df["account_discussion_creation_ratio"] = users_df["number_of_tweets"]/users_df["total_retweet_count"]
users_df['account_discussion_creation_ratio'].replace([np.inf, -np.inf], 0, inplace=True)
users_df

## 16. Ratio between num of tweets and num of likes

Creating the account ratio between num of tweets and num of likes

In [ ]:
users_df["tweet_num_likes_ratio"] = users_df["number_of_tweets"]/users_df["total_likes"]
users_df['tweet_num_likes_ratio'].replace([np.inf, -np.inf], 0, inplace=True)
users_df

## 17. Ratio between num of tweets and num of comments

Creating the account ratio between num of tweets and num of comments

In [ ]:
users_df["tweet_num_replies_ratio"] = users_df["number_of_tweets"]/users_df["total_replies"] 
users_df['tweet_num_replies_ratio'].replace([np.inf, -np.inf], 0, inplace=True)
users_df

## Cast to integers the values of the indicators for Users_df

In [ ]:
users_df['account_average_tweets_per_day'] = pd.to_numeric(users_df['account_average_tweets_per_day'], errors='coerce').astype(np.int64)
users_df['avg_tweets_per_actual_day'] = pd.to_numeric(users_df['avg_tweets_per_actual_day'], errors='coerce').astype(np.int64)
users_df['covid19_num_of_tweets'] = pd.to_numeric(users_df['covid19_num_of_tweets'], errors='coerce').astype(np.int64)
users_df['avg_hashtags'] = pd.to_numeric(users_df['avg_hashtags'], errors='coerce').astype(np.int64)
users_df['avg_text_length'] = pd.to_numeric(users_df['avg_text_length'], errors='coerce').astype(np.int64)
users_df['avg_mentions'] = pd.to_numeric(users_df['avg_mentions'], errors='coerce').astype(np.int64)
users_df['avg_special_char_in_text'] = pd.to_numeric(users_df['avg_special_char_in_text'], errors='coerce').astype(np.int64)
users_df['avt_favorite_count'] = pd.to_numeric(users_df['avt_favorite_count'], errors='coerce').astype(np.int64)
users_df['total_replies'] = pd.to_numeric(users_df['total_replies'], errors='coerce').astype(np.int64)
users_df['avt_reply_count'] = pd.to_numeric(users_df['avt_reply_count'], errors='coerce').astype(np.int64)
users_df['total_retweet_count'] = pd.to_numeric(users_df['total_retweet_count'], errors='coerce').astype(np.int64)
users_df['account_discussion_creation_ratio'] = pd.to_numeric(users_df['account_discussion_creation_ratio'], errors='coerce').astype(np.int64)
users_df['tweet_num_likes_ratio'] = pd.to_numeric(users_df['tweet_num_likes_ratio'], errors='coerce').astype(np.int64)
users_df['tweet_num_replies_ratio'] = pd.to_numeric(users_df['tweet_num_replies_ratio'], errors='coerce').astype(np.int64)

# Change Day with most tweet from num of days to actual day
users_df['day_with_most_tweets'] = datetime(2022,9,29) -  users_df['day_with_most_tweets'].apply(timedelta)

users_df.info(verbose=True, show_counts=True)
users_df.to_csv("./dataset/users_df_dataset_cleaned_with_indicators.csv",index=False) # Removes the counting of the Index rows

In [ ]:
indicator_columns_users = ['account_average_tweets_per_day', 'avg_tweets_per_actual_day', 'covid19_num_of_tweets', 'avg_hashtags', 
                            'avg_text_length', 'avg_mentions', 'avg_special_char_in_text', 'avt_favorite_count', 'total_replies',
                            'avt_reply_count', 'total_retweet_count', 'account_discussion_creation_ratio', 'tweet_num_likes_ratio',
                            'tweet_num_replies_ratio']

users_df[indicator_columns_users].plot(kind='box', figsize=(39,12), logy=True)


users_df_desc = users_df[indicator_columns_users].describe()
users_df_desc.to_csv('./dataset/users_df_describe_indicators.csv')

## Export merged_df with indicators

In [ ]:
merged_df.to_csv("./dataset/merged_df_dataset_cleaned_with_indicators.csv",index=False)

## Create a csv with number describing merged_df

In [ ]:
merged_df_desc = merged_df.describe()
merged_df_desc.drop(columns="user_id",inplace=True)
merged_df_desc.drop(columns="tweet_id",inplace=True)
merged_df_desc.drop(columns="bot",inplace=True)
merged_df_desc.to_csv('./dataset/merged_df_describe_indicators.csv')

In [ ]:
merged_df_only_indicators = merged_df[['account_average_tweets_per_day', 'text_length', 'special_char_in_text']]
merged_df_only_indicators.plot(kind='box', figsize=(9,4), logy=True)